In [67]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras import optimizers
from sklearn.metrics import hamming_loss
from sklearn import metrics
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score,average_precision_score
from keras.utils.vis_utils import plot_model
def metric_cal(y_test,yhat):
    yhat2 =yhat
    yhat2 = yhat2.round()
    
    mac_f1=f1_score(y_test, yhat2, average='macro')

    avg_percision=average_precision_score(y_test, yhat)
    hm_loss=hamming_loss(y_test, yhat2)
    print("MacF1:",mac_f1," avg_percision:",avg_percision, " Hamming loss:", hm_loss )
    
def drop_rare_label(df_train_y,df_test_y):
    print(df_train_y.shape)
  
    for label in range (df_train_y.shape[1]):

        if (df_train_y.loc[label].sum()<20 or a/df_train_y.loc[label].sum() >=50 ):
            df_train_y=df_train_y.drop(label , axis=1)
            df_test_y=df_test_y.drop(label , axis=1)
    print(df_train_y.shape)
    return df_train_y,df_test_y
def read_data(path):
    
    df_train = pd.read_csv(path+"train.csv")
    df_test = pd.read_csv(path+"test.csv")
    
    
    df_train_x=df_train.iloc[:,:-6]
    df_train_y=df_train.iloc[:,-6:]
    
    
    df_test_x=df_test.iloc[:,:-6]
    df_test_y=df_test.iloc[:,-6:]
    df_train_y,df_test_y=drop_rare_label(df_train_y,df_test_y)
    print(df_train.shape)
    
    return df_train_x,df_train_y,df_test_x,df_test_y




def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(64, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(16, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(8, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))

    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model









def evaluate_model(X_train,y_train, X_test,y_test):
    results = list()
    n_inputs, n_outputs = X_train.shape[1], y_train.shape[1]
    # define evaluation procedure
    #cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # enumerate folds

    # prepare data
    #X_train, X_test = X[train_ix], X[test_ix]
    #y_train, y_test = y[train_ix], y[test_ix]
    # define model
    model = get_model(n_inputs, n_outputs)
    # fit model
    model.fit(X_train, y_train, verbose=0, epochs=100)
    # make a prediction on the test set
    yhat = model.predict(X_test)
    # round probabilities to class labels
    yhat = yhat.round()
    # calculate accuracy
    acc = accuracy_score(y_test, yhat)
    # store result
    print('>%.3f' % acc)
    results.append(acc)
    return results,model,yhat

def Emotion_model(train_x,train_y,epochs, batch_size,Optimizer,output,weights):
    model = Sequential()
    LR=0.001
    Optimizer = optimizers.Adam(lr=0.001)
#    Optimizer = tf.optimizers.Adam(lr=0.001)
    dim_x,dim_y=train_x.shape
    
    model.add(Dense(32, input_dim=dim_y, activation='relu'))#294
 


    model.add(Dense(output, activation='relu'))
   
    x= Dense(output,   activation='relu' )
    model.add(x) 

   #x.set_weights(weights)
    #model.add(Dense(output, activation='sigmoid'))
    model.compile(loss=my_loss_fn, optimizer=Optimizer, metrics=["accuracy"])
    #model.compile(loss=my_loss_fn, optimizer=Optimizer, metrics=["accuracy"])

    X=train_x
    y=train_y
    #es_callback = keras.callbacks.EarlyStopping(monitor='loss', patience=5)
    #model.compile(optimizer='adam', loss=my_loss_fn)
    #model.fit(X, y,epochs, batch_size,callbacks=[es_callback])
    model.fit(X, y,epochs, batch_size,verbose=1) 
    
    return model

def probab(y_train):
    s = (y_train.shape[1],y_train.shape[1])
    PROB=np.zeros(s)
    
    for j in range (y_train.shape[1]):
        tmp=np.zeros(y_train.shape[1])
        count=0
        for i in range (y_train.shape[0]):
            tmp+=y_train.iloc[i].values*y_train.iloc[i].values[j]
            count+=y_train.iloc[i].values[j]
        tmp=tmp/count
        PROB[j]=tmp
            
    return (PROB)
    
    
import keras.backend as K   

##############################
def my_loss_fn(y_true, y_pred):
    
    Gy=tf.get_static_value(y_true)
    Py=tf.get_static_value(y_pred)
    PR=tf.convert_to_tensor(PROB.T, dtype=tf.float32)
    c=tf.matmul(y_pred,PR)
    #print(tf.get_static_value(z))
    print(c)
    casted = K.cast(y_true,"float")
    squared_difference=tf.square( (casted)-y_pred )
    squared_difference += tf.square(casted - c )
    #
    
    print("res::: ",tf.get_static_value(tf.reduce_mean(squared_difference, axis=-1)))
    return tf.reduce_mean(squared_difference, axis=-1)  # Note the `axis=-1`


X_train,y_train, X_test,y_test=read_data("../Birds/")
PROB=probab(y_train)


PR=tf.convert_to_tensor(
    PROB, dtype=tf.float32)
#results,model,yhat = evaluate_model(X_train,y_train, X_test,y_test)
model=Emotion_model(X_train,y_train,20,100,"Adam",y_train.shape[1],[PROB.T,PROB[0]*0])



yhat = model.predict(X_test)

print("yhat",yhat)

metric_cal(y_test,yhat)
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

(322, 6)


KeyError: '[0] not found in axis'

### 

In [27]:
model.layers[2].get_weights()

[array([[1.        , 0.3084112 , 0.05357143, 0.        , 0.04210526,
         0.46564886],
        [0.27731094, 1.        , 0.3452381 , 0.06741573, 0.01052632,
         0.04580153],
        [0.07563026, 0.5420561 , 1.        , 0.74157304, 0.6315789 ,
         0.03816794],
        [0.        , 0.05607476, 0.39285713, 1.        , 0.58947366,
         0.01526718],
        [0.03361345, 0.00934579, 0.35714287, 0.6292135 , 1.        ,
         0.06870229],
        [0.5126051 , 0.05607476, 0.0297619 , 0.02247191, 0.09473684,
         1.        ]], dtype=float32),
 array([0., 0., 0., 0., 0., 0.], dtype=float32)]

In [ ]:
!pip install torch
